<center> DL 심화 과제 제출_18기_수학과_신혜연 </center>

# 참고 사이트
 - https://medium.com/@inmoonlight/pytorch로-딥러닝하기-cnn-62a9326111ae
 - https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

# DL 심화 과제
MNIST test data에 대해 accuracy 98% 이상인 모델 만들어보기

In [46]:
import itertools
from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

%matplotlib inline

In [47]:
trn_dataset = datasets.MNIST('../mnist_data/',
                             download=True,
                             train=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(), # image to Tensor
                                 transforms.Normalize((0.1307,), (0.3081,)) # image, label
                             ])) 

val_dataset = datasets.MNIST("../mnist_data/", 
                             download=False,
                             train=False,
                             transform= transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307, ),(0.3081, ))
                           ]))

In [48]:
batch_size = 64
trn_loader = torch.utils.data.DataLoader(trn_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

In [49]:
# construct model on cuda if available
use_cuda = torch.cuda.is_available()

class CNNClassifier(nn.Module):
    
    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNNClassifier, self).__init__()
        conv1 = nn.Conv2d(1, 6, 5, 1) # 6@24*24
        # activation ReLU
        pool1 = nn.MaxPool2d(2) # 6@12*12
        conv2 = nn.Conv2d(6, 16, 5, 1) # 16@8*8
        # activation ReLU
        pool2 = nn.MaxPool2d(2) # 16@4*4
        
        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2
        )
        
        fc1 = nn.Linear(16*4*4, 240)
        # activation ReLU
        fc2 = nn.Linear(240, 84)
        # activation ReLU
        fc3 = nn.Linear(84, 10)

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.ReLU(),
            fc3
        )
        
        # gpu로 할당
        if use_cuda:
            self.conv_module = self.conv_module.cuda()
            self.fc_module = self.fc_module.cuda()
        
    def forward(self, x):
        out = self.conv_module(x) # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]: #16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return F.softmax(out, dim=1)

In [50]:
# 직접 만든CNNClassifier() 를 cnn = CNNClassifier() 로 선언하여 모델학습에 사용
cnn = CNNClassifier()

# loss
criterion = nn.CrossEntropyLoss()
# backpropagation method
learning_rate = 1e-3
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
# hyper-parameters
num_epochs = 5
num_batches = len(trn_loader)

trn_loss_list = []
val_loss_list = []
for epoch in range(num_epochs):
    trn_loss = 0.0
    for i, data in enumerate(trn_loader):
        x, label = data
        if use_cuda:
            x = x.cuda()
            label = label.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        model_output = cnn(x)
        # calculate loss
        loss = criterion(model_output, label)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del model_output
        
        # 학습과정 출력
        if (i+1) % 100 == 0: # every 100 mini-batches
            with torch.no_grad(): # very very very very important!!!
                val_loss = 0.0
                for j, val in enumerate(val_loader):
                    val_x, val_label = val
                    if use_cuda:
                        val_x = val_x.cuda()
                        val_label =val_label.cuda()
                    val_output = cnn(val_x)
                    v_loss = criterion(val_output, val_label)
                    val_loss += v_loss
                       
            print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
                epoch+1, num_epochs, i+1, num_batches, trn_loss / 100, val_loss / len(val_loader)
            ))            
            
            trn_loss_list.append(trn_loss/100)
            val_loss_list.append(val_loss/len(val_loader))
            trn_loss = 0.0

epoch: 1/5 | step: 100/938 | trn loss: 1.9211 | val loss: 1.7469
epoch: 1/5 | step: 200/938 | trn loss: 1.6572 | val loss: 1.6248
epoch: 1/5 | step: 300/938 | trn loss: 1.6082 | val loss: 1.5597
epoch: 1/5 | step: 400/938 | trn loss: 1.5355 | val loss: 1.5370
epoch: 1/5 | step: 500/938 | trn loss: 1.5264 | val loss: 1.5134
epoch: 1/5 | step: 600/938 | trn loss: 1.5181 | val loss: 1.5053
epoch: 1/5 | step: 700/938 | trn loss: 1.5102 | val loss: 1.5040
epoch: 1/5 | step: 800/938 | trn loss: 1.5036 | val loss: 1.4968
epoch: 1/5 | step: 900/938 | trn loss: 1.5069 | val loss: 1.4943
epoch: 2/5 | step: 100/938 | trn loss: 1.4968 | val loss: 1.4949
epoch: 2/5 | step: 200/938 | trn loss: 1.4956 | val loss: 1.4917
epoch: 2/5 | step: 300/938 | trn loss: 1.4964 | val loss: 1.4940
epoch: 2/5 | step: 400/938 | trn loss: 1.5000 | val loss: 1.4917
epoch: 2/5 | step: 500/938 | trn loss: 1.4948 | val loss: 1.4858
epoch: 2/5 | step: 600/938 | trn loss: 1.4902 | val loss: 1.5009
epoch: 2/5 | step: 700/93

In [51]:
# test
with torch.no_grad():
    X_test = val_dataset.data.view(len(val_dataset), 1, 28, 28).float()
    Y_test = val_dataset.targets
    
    prediction = cnn(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: {}'.format(accuracy.item()))

Accuracy: 0.973800003528595
